In [ ]:
import os
from io import StringIO

import sys

current_dir = os.getcwd()
sys.path.append(os.path.abspath(os.path.join(current_dir, '..', '..', 'PVT-thermal-performance-model')))
sys.path.append(os.path.abspath(os.path.join(current_dir, '..', '..', 'PVT-thermal-performance-model', 'CFD')))
sys.path.append(os.path.abspath(os.path.join(current_dir, '..', '..', 'RD-systems-and-test-benches')))
sys.path.append(os.path.abspath(os.path.join(current_dir, '..', '..', 'parallel-flow-distribution-pressure-loss')))
sys.path.append(os.path.abspath(os.path.join(current_dir, '..', '..', 'parallel-flow-distribution-pressure-loss', 'ansys')))

import numpy as np
import pandas as pd
import openpyxl

import matplotlib.pyplot as plt
import plotly.graph_objects as go

import jou_gen as jg
import bridge_module as bm

import ansys.fluent.core as pyfluent

from CoolProp.CoolProp import PropsSI

import hx_hydraulic as hxhy

In [ ]:
computer = 'seagull'
jg.set_computer(computer)
server_number = 3000

category_folder_name = 'Hydraulics'
subcategory_folder_name = 'V4.5'
project_folder_name = 'V4.5_1channel'

caoMeshFolder = 'V4.5_1chSide_M1'
title = 'parametric_laminar_try1'

nb_it = 1000

report_names = ['report_outlet_sp', 'report_inlet_sp', 'report_outlet_mdot']

In [ ]:
if computer == 'seagull':
    root_fp = r'D:\ANSYS Fluent Projects'
else:
    root_fp = "/usrtmp/delachaux/ANSYS Fluent Projects"
    jg.computer = computer

main_fp = os.path.join(root_fp, category_folder_name, subcategory_folder_name, project_folder_name)
Mesh_fp = os.path.join(main_fp, 'mesh')
SR_fp = bm.check_folder(os.path.join(main_fp, 'SimulationResults'))
caoMesh_fp = bm.check_folder(os.path.join(SR_fp, caoMeshFolder))
param_fp = bm.check_folder(os.path.join(SR_fp, caoMeshFolder, title))

tui, solver = bm.init_solver(main_fp, str(server_number), subfolder='')
print('main_fp :',main_fp)

In [ ]:
T_op = 25 + 273.15
p_op = 101325
print('density', PropsSI('D', 'T', T_op, 'P', p_op, 'INCOMP::MPG[0.4]'))
print('viscosity', PropsSI('V', 'T', T_op, 'P', p_op, 'INCOMP::MPG[0.4]'))

In [ ]:
import math

In [ ]:
Vdot = 100
Vdot_m3s = Vdot * 1e-3 / 3600 # m3/s
Ac = math.pi * ((0.0354/2)**2)
V = Vdot_m3s / Ac
rho = PropsSI('D', 'T', T_op, 'P', p_op, 'INCOMP::MPG[0.4]')
mdot = rho * Vdot_m3s
print('mdot', mdot)
print('V', V)

In [ ]:
water_dict = solver.setup.materials.fluid["water-liquid"].get_state()

T_op = 25 + 273.15
p_op = 101325

if computer == 'lmps_cds':
    constant_key = 'value'
    water_dict['chemical_formula'] = ""
else:
    constant_key = 'constant'

water_dict['density']['option'] = 'constant'
water_dict['density'][constant_key] = PropsSI('D', 'T', T_op, 'P', p_op, 'INCOMP::MPG[0.4]')
water_dict['viscosity']['option'] = 'constant'
water_dict['viscosity'][constant_key] = PropsSI('viscosity', 'T', T_op, 'P', p_op, 'INCOMP::MPG[0.4]')

solver.setup.materials.fluid["water-glycol"].set_state(water_dict)

In [ ]:
proj_x = 1.858884e-07
proj_y = -0.0192837
proj_z = -0.0160013

coeff_x = proj_x/0.02505798417538183
coeff_y = proj_y/0.02505798417538183
coeff_z = proj_z/0.02505798417538183

In [ ]:
# Vdot_list = [0.01, 0.05, 0.1, 0.5, 1., 1.5, 2., 3., 5.] 
Vdot_list = [0.01, 0.025, 0.05, 0.1, 0.5, 1., 1.5, 2., 3., 5.] 
# Vdot_list = [1., 1.5, 2., 3., 5.] 

In [ ]:
SR_fp

In [ ]:
for Vdot in Vdot_list:

    case_name = f'case_{str(Vdot)}Lh'
    folder_path_case = bm.check_folder(os.path.join(SR_fp, caoMeshFolder, title, case_name))

    for report_name in report_names:
        jg.change_report_file_path(tui, report_name, os.path.join(folder_path_case, f"{report_name}.out"))

    Vdot_m3s = Vdot * 1e-3 / 3600 # m3/s
    Ac = 0.0000055427 # m2
    V = Vdot_m3s / Ac # m/s

    jg.change_named_expression(tui, 'V_inlet', V, 'm/s')

    # if computer == 'seagull':
    #     channel = hxhy.duct('rectangular', h = 0.002, w = 0.0028, L = 1.603) # kPa
    #     PL_guess = 1000 * channel.regular_PL(Vdot, fluid_dict={'name':'MPG','glycol_rate':0.4}, p=p_op, T=T_op) # Pa
    #     jg.standard_initialization(tui, 'inlet', PL_guess, coeff_x*V, coeff_y*V, coeff_z*V)
    # else:
    solver.solution.initialization.hybrid_initialize()

    if computer == 'seagull':
        solver.solution.run_calculation.iterate(number_of_iterations = nb_it)
    elif computer == 'lmps_cds':
        solver.solution.run_calculation.iterate(iter_count = nb_it)

    jg.write_report(tui, 'mdot', folder_path_case, 'mdot_report')
    jg.write_report(tui, 'sp', folder_path_case, 'sp_report')
    jg.write_residuals_file(tui, folder_path_case, 'residuals')
    jg.write_data(tui, folder_path_case, f'case_{str(Vdot)}Lh')